In [2]:
import pandas as pd
import numpy as np 
import re
import nltk
#from transformers import BertTokenizer

### Create a setence level data set using the filename, speaker, and timestamp as labels 

In [3]:
file_path = '/Users/steffenerickson/Box Sync/ECR Observation Data/2023-2024 Final Data/Quant Team - Results'
df = pd.read_csv(f'{file_path}/transcripts.csv')

In [4]:
df_lines_split = df['text'].str.split(r'\\n').apply(pd.Series).stack().to_frame('line_str')
df_lines_split.reset_index(drop=True, inplace=True)
df_lines_split['filename'] = df['filename'].repeat(df_lines_split.shape[0] // len(df)).reset_index(drop=True)
df_cleaned = df_lines_split[df_lines_split['line_str'].str.strip() != ""]

In [5]:
def is_teacher_timestamp(text):
    return bool(re.search(r'teacher \d+.*\d+:\d+', text, re.IGNORECASE))
def is_transcribed_by_otter(text):
    return bool(re.search(r'transcribed by https://otter\.ai', text, re.IGNORECASE))

In [6]:
df_cleaned = df_cleaned.copy()
df_cleaned.loc[:, 'is_teacher_timestamp'] = df_cleaned['line_str'].apply(is_teacher_timestamp)
df_cleaned.loc[:, 'firstrow'] = df_cleaned.groupby('filename')['is_teacher_timestamp'].transform(
    lambda x: x.idxmax() == x.index).fillna(False).astype(int)
df_cleaned.loc[:, 'is_transcribed_by_otter'] = df_cleaned['line_str'].apply(is_transcribed_by_otter)
df_cleaned.loc[:, 'lastrow'] = df_cleaned['is_transcribed_by_otter'].apply(lambda x: 1 if x else 0)

In [7]:
def keep_between_ones(group):
    start_idx = group['firstrow'].idxmax()  # First occurrence of 1 in the second column
    end_idx = group['lastrow'].idxmax()    # First occurrence of 1 in the third column
    return group.loc[start_idx:end_idx-1]
df_chopped = df_cleaned.groupby('filename', group_keys=False).apply(keep_between_ones)
df_chopped = df_chopped.drop(['firstrow', 'lastrow','is_teacher_timestamp','is_transcribed_by_otter'], axis=1)

In [8]:
# Define a function to classify the rows based on the 'line_str'
def classify_line_str(line):
    if re.match(r'Teacher.*\d{2}:\d{2}', line):
        return 1
    elif re.match(r'.*\d{2}:\d{2}', line):
        return 2
    else:
        return np.nan
df_chopped['classification'] = df_chopped['line_str'].apply(classify_line_str)

In [9]:
# Define a function to extract the person (text before the timestamp) and timestamp
def extract_person_and_timestamp(line):
    # Use regex to match the format "[text] [timestamp]"
    match = re.search(r'(.+?)\s+(\d{2}:\d{2})', line)  # Adjusted to find any text before the timestamp
    if match:
        return match.group(1).strip(), match.group(2)
    else:
        return np.nan, np.nan
df_chopped['person'], df_chopped['timestamp'] = zip(*df_chopped.apply(
    lambda row: extract_person_and_timestamp(row['line_str']) if pd.notna(row['classification']) else (np.nan, np.nan), axis=1))

In [10]:
df_chopped['keep'] = df_chopped['classification'].apply(lambda x: 1 if pd.isna(x) else 0)
df_chopped['classification'] = df_chopped['classification'].ffill()
df_chopped['person'] = df_chopped['person'].ffill()
df_chopped['timestamp'] = df_chopped['timestamp'].ffill()
df_filtered = df_chopped[df_chopped['keep'] == 1]
df_filtered = df_filtered.drop(['keep'], axis=1)
df_filtered.reset_index(drop=True, inplace=True)

In [11]:
df_sentences = df_filtered.line_str.apply(lambda x: pd.Series(nltk.sent_tokenize(x))).stack().to_frame('sent_str')
df_sentences = df_sentences.reset_index(level=1, drop=True)
variables = ['filename', 'classification', 'person', 'timestamp','line_str']
for var in variables:
    df_sentences[var] = df_filtered[var].repeat(df_sentences.groupby(level=0).size()).values

In [12]:
#df_speaker = df_sentences[df_sentences['person'].str.contains('speaker', case=False, na=False)]

In [13]:
## Apply the conditions to locate the rows to be updated
#condition = (
#    (df_sentences['filename'] == "01_0101_G4_L03_transcript.txt") & 
#    (df_sentences['person'].str.contains('speaker', case=False, na=False))
#)
## Update the 'person' column
#df_sentences.loc[condition, 'person'] = "Teacher 01_0101"
## Update the 'classification' column to 1 for the same rows
#df_sentences.loc[condition, 'classification'] = 1

### Tokenize Sentences 

In [14]:
keep_whitespace = True
if keep_whitespace:
    # Return a tokenized copy of text
    # using NLTK's recommended word tokenizer.
    df_tokens = df_sentences.sent_str\
            .apply(lambda x: pd.Series(nltk.pos_tag(nltk.word_tokenize(x))))\
            .stack()\
            .to_frame('pos_tuple')
else:
    # Tokenize a string on whitespace (space, tab, newline).
    # In general, users should use the string ``split()`` method instead.
    # Returns fewer tokens.
    df_tokens = SENTS.sent_str\
            .apply(lambda x: pd.Series(nltk.pos_tag(nltk.WhitespaceTokenizer().tokenize(x))))\
            .stack()\
            .to_frame('pos_tuple')
df_tokens['pos'] = df_tokens.pos_tuple.apply(lambda x: x[1])
df_tokens['token_str'] = df_tokens.pos_tuple.apply(lambda x: x[0])
df_tokens['term_str'] = df_tokens.token_str.str.lower().str.replace(r"\W+", "", regex=True)
df_tokens['pos_group'] = df_tokens.pos.str[:2]

In [15]:
variables = ['filename', 'classification', 'person', 'timestamp','sent_str','line_str']
repeat_counts = df_sentences['sent_str'].apply(lambda x: len(nltk.word_tokenize(x)))
for var in variables:
    df_tokens[var] = np.repeat(df_sentences[var].values, repeat_counts)

In [16]:
df_tokens = df_tokens[df_tokens.term_str != '']

In [17]:
df_tokens.head(50)

pos_tuple  pos token_str term_str pos_group  \
0 0       (You, PRP)  PRP       You      you        PR   
  1      (could, MD)   MD     could    could        MD   
  2       (have, VB)   VB      have     have        VB   
  3     (wrote, VBD)  VBD     wrote    wrote        VB   
  4   (Jupiter, NNP)  NNP   Jupiter  jupiter        NN   
  6    (Saturn, NNP)  NNP    Saturn   saturn        NN   
  8    (Uranus, NNP)  NNP    Uranus   uranus        NN   
  9        (and, CC)   CC       and      and        CC   
  10  (Neptune, NNP)  NNP   Neptune  neptune        NN   
  0         (Or, CC)   CC        Or       or        CC   
1 0       (Gas, NNP)  NNP       Gas      gas        NN   
  1    (giants, NNS)  NNS    giants   giants        NN   
2 0       (you, PRP)  PRP       you      you        PR   
  1      (could, MD)   MD     could    could        MD   
  2       (have, VB)   VB      have     have        VB   
  3       (just, RB)   RB      just     just        RB   
  4     (wrote, VBD)  VBD     wrote    wrote        VB   
  5         (in, IN)   IN        in       in        IN   
  6        (gas, NN)   NN       gas      gas        NN   
  7    (giants, NNS)  NNS    giants   giants        NN   
  0        (All, DT)   DT       All      all        DT   
  1         (of, IN)   IN        of       of        IN   
  2      (those, DT)   DT     those    those        DT   
  3        (gas, NN)   NN       gas      gas        NN   
  4    (giants, NNS)  NNS    giants   giants        NN   
  5      (have, VBP)  VBP      have     have        VB   
  6     (rings, NNS)  NNS     rings    rings        NN   
  8   (Jupiter, NNP)  NNP   Jupiter  jupiter        NN   
  10   (Saturn, NNP)  NNP    Saturn   saturn        NN   
  12   (Uranus, NNP)  NNP    Uranus   uranus        NN   
  13       (and, CC)   CC       and      and        CC   
  14  (Neptune, NNP)  NNP   Neptune  neptune        NN   
  0        (It, PRP)  PRP        It       it        PR   
  1        (is, VBZ)  VBZ        is       is        VB   
  2     (almost, RB)   RB    almost   almost        RB   
  3       (time, NN)   NN      time     time        NN   
  4         (to, TO)   TO        to       to        TO   
  5         (go, VB)   VB        go       go        VB   
  6         (so, RB)   RB        so       so        RB   
  7         (I, PRP)  PRP         I        i        PR   
  8        ('m, VBP)  VBP        'm        m        VB   
  9       (gon, VBG)  VBG       gon      gon        VB   
  10        (na, TO)   TO        na       na        TO   
  11      (move, VB)   VB      move     move        VB   
  12       (the, DT)   DT       the      the        DT   
  13    (screen, NN)   NN    screen   screen        NN   
  0         (If, IN)   IN        If       if        IN   
  1       (you, PRP)  PRP       you      you        PR   
  2      (have, VBP)  VBP      have     have        VB   
  3        (not, RB)   RB       not      not        RB   

                           filename  classification                person  \
0 0   01_0101_G4_L03_transcript.txt             1.0       Teacher 01_0101   
  1   01_0101_G4_L03_transcript.txt             1.0       Teacher 01_0101   
  2   01_0101_G4_L03_transcript.txt             1.0       Teacher 01_0101   
  3   01_0101_G4_L03_transcript.txt             1.0       Teacher 01_0101   
  4   01_0101_G4_L03_transcript.txt             1.0       Teacher 01_0101   
  6   01_0101_G4_L03_transcript.txt             1.0       Teacher 01_0101   
  8   01_0101_G4_L03_transcript.txt             1.0       Teacher 01_0101   
  9   01_0101_G4_L03_transcript.txt             1.0       Teacher 01_0101   
  10  01_0101_G4_L03_transcript.txt             1.0       Teacher 01_0101   
  0   01_0101_G4_L03_transcript.txt             1.0       Teacher 01_0101   
1 0   01_0101_G4_L03_transcript.txt             2.0  Unidentified Student   
  1   01_0101_G4_L03_transcript.txt             2.0  Unidentified Student   
2 0   01_0101_G4_L03_transcript.txt             1.0

### Vocab Tables 

In [18]:
# Teacher Vocab 
df_tokens_teacher = df_tokens[df_tokens['classification'] == 1]
vocab_teacher = df_tokens_teacher['term_str'].value_counts().to_frame('n')
vocab_teacher.index.name = 'term_str'
vocab_teacher['p'] = vocab_teacher['n'] / vocab_teacher['n'].sum()  # Probability of each term
vocab_teacher['i'] = -np.log2(vocab_teacher['p'])  # Information content
vocab_teacher['n_chars'] = vocab_teacher.index.str.len()  # Number of characters in each term

In [19]:
#Student Vocab
df_tokens_student = df_tokens[df_tokens['classification'] == 2]
vocab_student = df_tokens_student['term_str'].value_counts().to_frame('n')
vocab_student.index.name = 'term_str'
vocab_student['p'] = vocab_student['n'] / vocab_student['n'].sum()  # Probability of each term
vocab_student['i'] = -np.log2(vocab_student['p'])  # Information content
vocab_student['n_chars'] = vocab_student.index.str.len()  # Number of characters in each term
vocab_student.head()

,n,p,i,n_chars
term_str,,,,
the,1123,0.042482,4.557019,3
it,872,0.032987,4.921977,2
i,640,0.024210,5.368234,1
and,589,0.022281,5.488038,3
a,537,0.020314,5.621383,1


In [20]:
vocab_teacher.head(20)

,n,p,i,n_chars
term_str,,,,
you,5355,0.042030,4.572437,3
the,4040,0.031709,4.978968,3
to,3381,0.026537,5.235873,2
it,3316,0.026026,5.263879,2
i,2649,0.020791,5.587876,1
and,2508,0.019685,5.666786,3
that,2321,0.018217,5.778577,4
a,2031,0.015941,5.971133,1
s,1999,0.015690,5.994045,1


In [21]:
vocab_student.head(20)

,n,p,i,n_chars
term_str,,,,
the,1123,0.042482,4.557019,3
it,872,0.032987,4.921977,2
i,640,0.024210,5.368234,1
and,589,0.022281,5.488038,3
a,537,0.020314,5.621383,1
you,514,0.019444,5.684537,3
to,502,0.018990,5.718618,2
s,443,0.016758,5.898999,1
that,403,0.015245,6.035526,4


In [22]:
df_tokens_student.head(50)

pos_tuple  pos    token_str     term_str pos_group  \
1  0           (Gas, NNP)  NNP          Gas          gas        NN   
   1        (giants, NNS)  NNS       giants       giants        NN   
6  0             (I, PRP)  PRP            I            i        PR   
   1            ('m, VBP)  VBP           'm            m        VB   
   2        (trying, VBG)  VBG       trying       trying        VB   
   3             (to, TO)   TO           to           to        TO   
   4          (think, VB)   VB        think        think        VB   
   5             (of, IN)   IN           of           of        IN   
   6            (the, DT)   DT          the          the        DT   
   7           (song, NN)   NN         song         song        NN   
   8         (where, WRB)  WRB        where        where        WR   
   9            (it, PRP)  PRP           it           it        PR   
   10        (sings, VBZ)  VBZ        sings        sings        VB   
   12            (I, PRP)  PRP            I            i        PR   
   13           ('m, VBP)  VBP           'm            m        VB   
   14       (trying, VBG)  VBG       trying       trying        VB   
   15            (to, TO)   TO           to           to        TO   
   16         (think, VB)   VB        think        think        VB   
   17          (what, WP)   WP         what         what        WP   
   18          (was, VBD)  VBD          was          was        VB   
   19        (before, IN)   IN       before       before        IN   
   20           (it, PRP)  PRP           it           it        PR   
   21           (and, CC)   CC          and          and        CC   
   22          (then, RB)   RB         then         then        RB   
9  0          (Wait, NNP)  NNP         Wait         wait        NN   
   1           (why, WRB)  WRB          why          why        WR   
   2           (did, VBD)  VBD          did          did        VB   
   3            (he, PRP)  PRP           he           he        PR   
   4            (lie, VB)   VB          lie          lie        VB   
10 0            (He, PRP)  PRP           He           he        PR   
   1          (said, VBD)  VBD         said         said        VB   
   2            (he, PRP)  PRP           he           he        PR   
   3          (said, VBD)  VBD         said         said        VB   
   4            (it, PRP)  PRP           it           it        PR   
   5           (was, VBD)  VBD          was          was        VB   
   6          (pride, RB)   RB        pride        pride        RB   
   0            (And, CC)   CC          And          and        CC   
   1           (then, RB)   RB         then         then        RB   
   2            (he, PRP)  PRP           he           he        PR   
   3          (said, VBD)  VBD         said         said        VB   
   4           (okay, NN)   NN         okay         okay        NN   
   5             (I, PRP)  PRP            I            i        PR   
   6          (lied, VBD)  VBD         lied         lied        VB   
   8        (because, IN)   IN      because      because        IN   
   9            (he, PRP)  PRP           he           he        PR   
   10           ('s, VBZ)  VBZ           's            s        VB   
   11       (because, IN)   IN      because      because        IN   
   12           (he, PRP)  PRP           he           he        PR   
   13           ('s, VBZ)  VBZ           's            s        VB   
   14  (embarrassed, VBN)  VBN  embarrassed  embarrassed        VB   

                            filename  classification                person  \
1  0   01_0101_G4_L03_transcript.txt             2.0  Unidentified Student   
   1   01_0101_G4_L03_transcript.txt             2.0  Unidentified Student   
6  0   01_0101_G4_L04_transcript.txt             2.0  Unidentified Student   
   1   01_0101_G4_L04_transcript.txt             2.0  Unidentified Student   
   2   01_0101_G4_L04_transcript.txt             2.0  U

In [23]:
df_tokens_teacher.head(50)

pos_tuple  pos token_str  term_str pos_group  \
0 0        (You, PRP)  PRP       You       you        PR   
  1       (could, MD)   MD     could     could        MD   
  2        (have, VB)   VB      have      have        VB   
  3      (wrote, VBD)  VBD     wrote     wrote        VB   
  4    (Jupiter, NNP)  NNP   Jupiter   jupiter        NN   
  6     (Saturn, NNP)  NNP    Saturn    saturn        NN   
  8     (Uranus, NNP)  NNP    Uranus    uranus        NN   
  9         (and, CC)   CC       and       and        CC   
  10   (Neptune, NNP)  NNP   Neptune   neptune        NN   
  0          (Or, CC)   CC        Or        or        CC   
2 0        (you, PRP)  PRP       you       you        PR   
  1       (could, MD)   MD     could     could        MD   
  2        (have, VB)   VB      have      have        VB   
  3        (just, RB)   RB      just      just        RB   
  4      (wrote, VBD)  VBD     wrote     wrote        VB   
  5          (in, IN)   IN        in        in        IN   
  6         (gas, NN)   NN       gas       gas        NN   
  7     (giants, NNS)  NNS    giants    giants        NN   
  0         (All, DT)   DT       All       all        DT   
  1          (of, IN)   IN        of        of        IN   
  2       (those, DT)   DT     those     those        DT   
  3         (gas, NN)   NN       gas       gas        NN   
  4     (giants, NNS)  NNS    giants    giants        NN   
  5       (have, VBP)  VBP      have      have        VB   
  6      (rings, NNS)  NNS     rings     rings        NN   
  8    (Jupiter, NNP)  NNP   Jupiter   jupiter        NN   
  10    (Saturn, NNP)  NNP    Saturn    saturn        NN   
  12    (Uranus, NNP)  NNP    Uranus    uranus        NN   
  13        (and, CC)   CC       and       and        CC   
  14   (Neptune, NNP)  NNP   Neptune   neptune        NN   
  0         (It, PRP)  PRP        It        it        PR   
  1         (is, VBZ)  VBZ        is        is        VB   
  2      (almost, RB)   RB    almost    almost        RB   
  3        (time, NN)   NN      time      time        NN   
  4          (to, TO)   TO        to        to        TO   
  5          (go, VB)   VB        go        go        VB   
  6          (so, RB)   RB        so        so        RB   
  7          (I, PRP)  PRP         I         i        PR   
  8         ('m, VBP)  VBP        'm         m        VB   
  9        (gon, VBG)  VBG       gon       gon        VB   
  10         (na, TO)   TO        na        na        TO   
  11       (move, VB)   VB      move      move        VB   
  12        (the, DT)   DT       the       the        DT   
  13     (screen, NN)   NN    screen    screen        NN   
  0          (If, IN)   IN        If        if        IN   
  1        (you, PRP)  PRP       you       you        PR   
  2       (have, VBP)  VBP      have      have        VB   
  3         (not, RB)   RB       not       not        RB   
  4   (finished, VBN)  VBN  finished  finished        VB   
  6        (that, DT)   DT      that      that        DT   

                           filename  classification           person  \
0 0   01_0101_G4_L03_transcript.txt             1.0  Teacher 01_0101   
  1   01_0101_G4_L03_transcript.txt             1.0  Teacher 01_0101   
  2   01_0101_G4_L03_transcript.txt             1.0  Teacher 01_0101   
  3   01_0101_G4_L03_transcript.txt             1.0  Teacher 01_0101   
  4   01_0101_G4_L03_transcript.txt             1.0  Teacher 01_0101   
  6   01_0101_G4_L03_transcript.txt             1.0  Teacher 01_0101   
  8   01_0101_G4_L03_transcript.txt             1.0  Teacher 01_0101   
  9   01_0101_G4_L03_transcript.txt             1.0  Teacher 01_0101   
  10  01_0101_G4_L03_transcript.txt             1.0  Teacher 01_0101   
  0   01_0101_G4_L03_transcript.txt             1.0  Teacher 01_0101   
2 0   01_0101_G4_L03_transcript.txt             1.0  Teacher 01_0101   
  1   01_0101_G4_L03_transcript.txt             1.0  Teacher 01_0101   
  2   01_0101_G

In [24]:
df_tokens.to_csv(f'{file_path}/df_tokens.csv', index=False)